#### [ 모델 저장 및 로딩 ]
- 2가지 형태 저장
    - 전체 저장
    - 모델의 파라미터만 저장

- 2가지 형태 로딩
    - 전체 저장 모델 파일 ==> 로딩으로 사용 가능
    - 모델 파라미터만 저장 ==> 모델 객체 생성 후 층별 파라미터 삽입


[1] 모듈 로딩 및 데이터 준비<hr>

In [100]:
## [1-1] 모듈 로딩
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import sys
sys.path.append("../Utils")

import utils as uf


In [101]:
## [1-2] 데이터 준비
df = pd.read_csv('../Data/study_score_multi.csv')

feature_cols = ["study_hours", "sleep_hours", "participation"]
target_col = "score"

## 텐서 변환
featureTS = torch.tensor(df[feature_cols].values, dtype=torch.float32)          # [N, 3]
targetTS = torch.tensor(df[[target_col]].values, dtype=torch.float32)          # [N, 1]

[2] 모델 클래스 정의 <hr>

In [102]:
class TEST(nn.Module):
    def __init__(self) :
        super().__init__()
        self.fc1 = nn.Linear(3, 16)
        self.fc2 = nn.Linear(16, 4)
        self.out = nn.Linear(4, 1)
        
    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = F.relu(self.fc2(out))
        return self.out(out)

In [103]:
## 설정값들
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
EPOCHS = 10
BS = 16
LR = 1e-3

## 저장 모델 파일명
ALL_MODEL = '../Models/all_model.pt'    ## 모델 전체 확장자
WEIGHTS_MODEL = '../Models/weights'     ## 파라미터 저장 확장자 pth



In [104]:

## 인스턴스들
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

model = TEST().to(DEVICE)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr =LR)

scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3)

## train, valid 분리 
X_train, X_valid, y_train, y_valid = train_test_split(
                                    featureTS,
                                    targetTS,
                                    test_size=0.2,     # validation 비율
                                    random_state=42,   # 재현성
                                    shuffle=True
)


train_ds = TensorDataset(X_train, y_train)
valid_ds = TensorDataset(X_valid, y_valid)

train_dl = DataLoader(train_ds, batch_size=BS, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=BS, shuffle=False)

In [105]:
for x, y in train_dl :
    print(x, y, sep='\n')
    break
print()
for x, y in valid_dl :
    print(x, y, sep='\n')
    break

tensor([[ 3.6200,  7.5700, 78.7000],
        [ 3.7300,  7.2700, 54.1000],
        [ 2.7600,  7.3000, 73.6000],
        [ 0.8700,  5.5300, 20.1000],
        [ 3.3300,  6.9600, 58.4000],
        [ 3.7900,  6.3800, 52.8000],
        [ 6.4800,  8.0800, 89.6000],
        [ 7.4500,  8.7300, 52.0000],
        [ 9.8700,  5.3400, 25.2000],
        [ 8.4200,  4.6900, 24.9000],
        [ 7.5600,  7.2500, 56.6000],
        [ 2.8300,  6.3400, 16.2000],
        [ 7.7000,  5.1000, 55.6000],
        [ 7.1300,  8.5700, 45.2000],
        [ 2.4600,  4.2600, 80.7000],
        [ 3.7100,  7.7600, 53.5000]])
tensor([[47.5000],
        [49.0000],
        [43.7000],
        [17.0000],
        [41.1000],
        [50.8000],
        [81.9000],
        [66.6000],
        [75.8000],
        [70.0000],
        [71.6000],
        [36.7000],
        [69.5000],
        [74.4000],
        [47.4000],
        [41.9000]])

tensor([[3.9400e+00, 8.9800e+00, 8.2000e+01],
        [7.2100e+00, 7.7700e+00, 1.7100e+01],
        [

In [106]:
import math
## 학습 진행 ==================
BEST_LOSS = 100.
EARLY_STOP_CNT = 10
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
for e in range(EPOCHS) :
    ## 학습진행
    train_loss, train_acc = uf.train_one_epoch(model,
                                               train_dl,
                                               loss_fn,
                                               optimizer,
                                               DEVICE)
    
    ## 검증 진행
    valid_loss, valid_acc = uf.evaluate(model, valid_dl, loss_fn, DEVICE)
    
    
    # 3. 회귀용 지표 계산 (MAE / RMSE)
    model.eval()

    train_mae, train_cnt = 0.0, 0
    valid_mae, valid_cnt = 0.0, 0

    with torch.no_grad():
        # --- train MAE ---
        for xb, yb in train_dl:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            pred = model(xb)
            train_mae += torch.abs(pred - yb).sum().item()
            train_cnt += yb.numel()

        # --- valid MAE ---
        for xb, yb in valid_dl:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            pred = model(xb)
            valid_mae += torch.abs(pred - yb).sum().item()
            valid_cnt += yb.numel()

    train_mae /= train_cnt
    valid_mae /= valid_cnt

    train_rmse = math.sqrt(train_loss)
    valid_rmse = math.sqrt(valid_loss)
    
    
    
    #- 모델과 가중치 파일 저장
    if BEST_LOSS > valid_loss : 
        ## 모델 전체 저장
        torch.save(model, ALL_MODEL)
        ## 파라미터만 저장
        torch.save(model.state_dict(), f"./{WEIGHTS_MODEL}_{e}.pth")
        ## 기준 loss 업데이트
        BEST_LOSS = valid_loss
        
    ## - 스케줄러에게 검증 성능 업데이트
    scheduler.step(valid_rmse)
    # =====================
    # 5. 출력 (회귀용)
    # =====================
    print(
        f"{e}. "
        f"[MSE] train:valid = {train_loss:.4f} : {valid_loss:.4f} | "
        # f"[RMSE] train:valid = {train_rmse:.4f} : {valid_rmse:.4f} | "
        # f"[MAE] train:valid = {train_mae:.4f} : {valid_mae:.4f}"
        f"[LR] {scheduler.get_last_lr()[0]:.4f} bad_epochs :  {scheduler.num_bad_epochs}"
        
    )
    # # 학습상태 출력 
    # print(f'{e}.[loss] train : valid = {train_loss} : {valid_loss} [acc] train : valid = {train_acc} : {valid_acc}')
    # print(f"[LR] {scheduler.get_last_lr()[0]:.4f} bad_epochs :  {scheduler.num_bad_epochs}")
    
    
    ## 조기 종료 체크
    if(scheduler.patience == scheduler.num_bad_epochs):
        EARLY_STOP_CNT -=1
    if not EARLY_STOP_CNT :
        print("성능 개선이 없어 조기종료 합니다.")
        break
    
print(EARLY_STOP_CNT)

0. [MSE] train:valid = 1517.6818 : 583.2661 | [LR] 0.0010 bad_epochs :  0
1. [MSE] train:valid = 495.8071 : 405.5073 | [LR] 0.0010 bad_epochs :  0
2. [MSE] train:valid = 310.0968 : 224.3832 | [LR] 0.0010 bad_epochs :  0
3. [MSE] train:valid = 159.4333 : 114.6714 | [LR] 0.0010 bad_epochs :  0
4. [MSE] train:valid = 90.5495 : 71.5268 | [LR] 0.0010 bad_epochs :  0
5. [MSE] train:valid = 58.8976 : 48.8763 | [LR] 0.0010 bad_epochs :  0
6. [MSE] train:valid = 44.6460 : 40.2402 | [LR] 0.0010 bad_epochs :  0
7. [MSE] train:valid = 39.5070 : 37.7035 | [LR] 0.0010 bad_epochs :  0
8. [MSE] train:valid = 38.0235 : 37.1192 | [LR] 0.0010 bad_epochs :  0
9. [MSE] train:valid = 37.7358 : 37.0169 | [LR] 0.0010 bad_epochs :  0
10


[4] 모델 파일 사용 <hr>

In [107]:
## [4-1] 가중치 저장 파일 로딩
params = torch.load("../Models/weights_3.pth", weights_only=True)

tModel = TEST()
tModel.load_state_dict(params)

<All keys matched successfully>

In [108]:
## [4-2] 전체 모델 저장 파일 로딩
allModel = torch.load(ALL_MODEL, weights_only=False)
allModel

TEST(
  (fc1): Linear(in_features=3, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=4, bias=True)
  (out): Linear(in_features=4, out_features=1, bias=True)
)